In [1]:
import requests
import json

import openstreetmap_mapping as osm

import pandas as pd

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
class Tags(object):
    
    def __init__(self, *initial_data, **kwargs):
              
        for dictionary in initial_data:
            
            for key in dictionary:

                if isinstance(dictionary[key], dict):
                    if len(dictionary[key])>1:
                        setattr(self, 'value', key)
                        setattr(self, key, Tags(dictionary[key]))
                    else:
                        setattr(self, key, Tags(dictionary[key]))
                else:
                    setattr(self, key, dictionary[key])
            
    def __repr__(self):
        return self.value.replace("__",":")
    
    def __str__(self):
        return self.value.replace("__",":")

In [4]:
kvs = osm.toolkit.get_osm_kvs()

In [5]:
tags = Tags(kvs)

In [6]:
area_Glasgow = "(55.82, -4.34, 55.9, -4.22)"
area_London = "(51.448, -0.2554, 51.6061, 0.0714)"

In [7]:
df_stations = osm.toolkit.get_osm_data(key=tags.railway,tag=tags.railway.station,area=area_Glasgow)


In [8]:
df_railway = osm.toolkit.get_osm_data(key=tags.railway,tag=tags.railway.rail,area=area_Glasgow,output='geom')


In [9]:
df_railway

,type,id,bounds,nodes,geometry,tags,electrified,frequency,gauge,maxspeed,...,tracks,service,embankment,NHD:Elevation,siding,fixme,lat,lon,key,tag
0,way,3413,"{'minlat': 55.8824801, 'minlon': -4.3256055, '...","[25812260, 2958370074, 2958370073, 4293930827,...","[{'lat': 55.8827057, 'lon': -4.3256055}, {'lat...","{'electrified': 'contact_line', 'frequency': '...",contact_line,50,1435,50 mph,...,NaN,NaN,NaN,NaN,NaN,NaN,55.882623,-4.324541,railway,rail
1,way,3417,"{'minlat': 55.8709381, 'minlon': -4.3140597, '...","[26362673, 1983068732, 2958368618, 702443, 295...","[{'lat': 55.8776742, 'lon': -4.3138852}, {'lat...","{'electrified': 'contact_line', 'frequency': '...",contact_line,50,1435,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55.874728,-4.313023,railway,rail
2,way,4110536,"{'minlat': 55.8586301, 'minlon': -4.2589441, '...","[264483615, 6888766152]","[{'lat': 55.8587282, 'lon': -4.2589439}, {'lat...","{'electrified': 'contact_line', 'frequency': '...",contact_line,50,1435,15 mph,...,NaN,NaN,NaN,NaN,NaN,NaN,55.858679,-4.258944,railway,rail
3,way,4110773,"{'minlat': 55.8460756, 'minlon': -4.2987346, '...","[30671370, 1646877767, 512641848, 2984394846, ...","[{'lat': 55.8460756, 'lon': -4.2987346}, {'lat...","{'electrified': 'contact_line', 'frequency': '...",contact_line,50,1435,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55.847110,-4.294095,railway,rail
4,way,4281149,"{'minlat': 55.8827057, 'minlon': -4.3264013, '...","[25812262, 25812260]","[{'lat': 55.8827252, 'lon': -4.3264013}, {'lat...","{'bridge': 'yes', 'electrified': 'contact_line...",contact_line,50,1435,50 mph,...,NaN,NaN,NaN,NaN,NaN,NaN,55.882715,-4.326003,railway,rail
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,way,909791084,"{'minlat': 55.8621843, 'minlon': -4.2512598, '...","[1195768695, 4294143074]","[{'lat': 55.8621843, 'lon': -4.2512598}, {'lat...","{'electrified': 'contact_line', 'frequency': '...",contact_line,50,1435,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55.862273,-4.251220,railway,rail
838,way,946666807,"{'minlat': 55.8254562, 'minlon': -4.2882563, '...","[3800316508, 8763075137]","[{'lat': 55.8255544, 'lon': -4.2882563}, {'lat...","{'bridge': 'yes', 'electrified': 'contact_line...",contact_line,50,1435,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55.825505,-4.288177,railway,rail
839,way,946666808,"{'minlat': 55.825449, 'minlon': -4.2883728, 'm...","[3800316500, 8763075138]","[{'lat': 55.8255485, 'lon': -4.2883728}, {'lat...","{'bridge': 'yes', 'electrified': 'contact_line...",contact_line,50,1435,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55.825499,-4.288306,railway,rail
840,way,946666816,"{'minlat': 55.8255544, 'minlon': -4.2883655, '...","[8763075205, 3800316508]","[{'lat': 55.8256211, 'lon': -4.2883655}, {'lat...","{'electrified': 'contact_line', 'frequency': '...",contact_line,50,1435,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,55.825588,-4.288311,railway,rail


In [17]:
pd.DataFrame.from_records(df_railway['geometry'])

,0,1,2,3,4,5,6,7,8,9,...,61,62,63,64,65,66,67,68,69,70
0,"{'lat': 55.8827057, 'lon': -4.3256055}","{'lat': 55.8826972, 'lon': -4.3254157}","{'lat': 55.8826921, 'lon': -4.325185}","{'lat': 55.8826483, 'lon': -4.3245628}","{'lat': 55.8825958, 'lon': -4.3240641}","{'lat': 55.8825411, 'lon': -4.3236462}","{'lat': 55.8824801, 'lon': -4.3233099}",None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"{'lat': 55.8776742, 'lon': -4.3138852}","{'lat': 55.8769942, 'lon': -4.3139865}","{'lat': 55.8766164, 'lon': -4.3140425}","{'lat': 55.876234, 'lon': -4.3140597}","{'lat': 55.8759419, 'lon': -4.3140181}","{'lat': 55.8756477, 'lon': -4.3139507}","{'lat': 55.8753504, 'lon': -4.3138007}","{'lat': 55.8750303, 'lon': -4.3135865}","{'lat': 55.8746771, 'lon': -4.3132969}","{'lat': 55.8743235, 'lon': -4.31294}",...,None,None,None,None,None,None,None,None,None,None
2,"{'lat': 55.8587282, 'lon': -4.2589439}","{'lat': 55.8586301, 'lon': -4.2589441}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"{'lat': 55.8460756, 'lon': -4.2987346}","{'lat': 55.8461256, 'lon': -4.2986269}","{'lat': 55.8464301, 'lon': -4.2979031}","{'lat': 55.8465178, 'lon': -4.2976949}","{'lat': 55.8466011, 'lon': -4.2974832}","{'lat': 55.8466857, 'lon': -4.2972537}","{'lat': 55.8467902, 'lon': -4.2969257}","{'lat': 55.8468672, 'lon': -4.2966855}","{'lat': 55.8469266, 'lon': -4.2964707}","{'lat': 55.8469829, 'lon': -4.2962626}",...,None,None,None,None,None,None,None,None,None,None
4,"{'lat': 55.8827252, 'lon': -4.3264013}","{'lat': 55.8827057, 'lon': -4.3256055}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,"{'lat': 55.8621843, 'lon': -4.2512598}","{'lat': 55.8623626, 'lon': -4.2511793}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
838,"{'lat': 55.8255544, 'lon': -4.2882563}","{'lat': 55.8254562, 'lon': -4.2880972}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
839,"{'lat': 55.8255485, 'lon': -4.2883728}","{'lat': 55.825449, 'lon': -4.2882399}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
840,"{'lat': 55.8256211, 'lon': -4.2883655}","{'lat': 55.8255544, 'lon': -4.2882563}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [10]:
# comment out this to revert to stations
df_stations = df_railway

In [11]:
#df_stations["grouping"] = df_stations["network"]
df_stations["grouping"] = df_stations["electrified"]

In [12]:
df_stations['grouping'] = df_stations['grouping'].str.split(";",expand=True)[0]

In [13]:
df_stations = df_stations.dropna(subset=['grouping'])

In [14]:
output_file("Glasgow_Stations.html")

tooltips = {"tooltips":[("id", "@id"),("key", "@key"),("tag", "@tag"),("name", "@name"),("network", "@network"),("operator", "@operator"),("(lat,lon)", "@coordinates")]}

p = osm.toolkit.plot_latlon(df_stations,kwargs_for_figure=tooltips,kwargs_for_marker={"legend_group":"grouping"})

p.legend.visible = False

show(p)